# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("./city_weather.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Ushuaia,-54.8000,-68.3000,35.26,93,100,25.32,AR,1661406053
1,1,Kapaa,22.0752,-159.3190,82.38,83,100,8.05,US,1661406128
2,2,Quatre Cocos,-20.2078,57.7625,73.51,70,53,16.33,MU,1661406128
3,3,Rikitea,-23.1203,-134.9692,69.19,59,56,16.17,PF,1661406129
4,4,Nuuk,64.1835,-51.7216,45.21,87,75,24.16,GL,1661406129


In [7]:
weather_df.count()

Unnamed: 0    585
City          585
Lat           585
Lng           585
Max Temp      585
Humidity      585
Cloudiness    585
Wind Speed    585
Country       583
Date          585
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
heatmap_humidity = gmaps.figure(map_type="HYBRID")
max_int = weather_df["Humidity"].max()
heat_layer = gmaps.heatmap_layer(locations=weather_df[['Lat', 'Lng']], weights=weather_df['Humidity'].astype(float), dissipating=False,
    max_intensity=max_int, point_radius=1)
heatmap_humidity.add_layer(heat_layer)
heatmap_humidity

100


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Weather conditions:
# Max Temp: between 65 and 70 degrees
# Wind Speed: 10 mph
# Cloudiness: 40-50%
perfect_weather = weather_df.loc[(weather_df["Max Temp"] >=60)&
    (weather_df["Max Temp"]<=70)&
    (weather_df["Wind Speed"]<=10)&
    (weather_df["Cloudiness"]<=50)&
    (weather_df["Cloudiness"]>=20), :]
perfect_weather.reset_index(drop=True, inplace=True)
perfect_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,140,Caravelas,-17.7125,-39.2481,67.39,92,21,5.08,BR,1661406189
1,173,Camacha,33.0833,-16.3333,67.42,83,40,9.22,PT,1661406205
2,227,Amapá,1.0000,-52.0000,69.40,99,44,1.52,BR,1661406228
3,235,Maxixe,-23.8597,35.3472,64.22,94,20,0.00,MZ,1661406232
4,276,Ipirá,-12.1583,-39.7372,63.10,89,43,4.09,BR,1661406250


In [22]:
perfect_weather.count()

Unnamed: 0    16
City          16
Lat           16
Lng           16
Max Temp      16
Humidity      16
Cloudiness    16
Wind Speed    16
Country       16
Date          16
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
hotel_df = perfect_weather.copy()
hotel_df["Hotel Name"]=""
hotel_df.columns

Index(['Unnamed: 0', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity',
       'Cloudiness', 'Wind Speed', 'Country', 'Date', 'Hotel Name'],
      dtype='object')

In [30]:
for r in range(0, len(hotel_df)):
    h_lat = hotel_df["Lat"][r]
    h_lng = hotel_df["Lng"][r]
    hotel_coord = f"{h_lat},{h_lng}"
    params = {
        "location": hotel_coord,
        "keyword": "hotel",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
    }
    response=requests.get("https://maps.googleapis.com/maps/api/place/nearbysearch/json", params=params).json()
    hotel_df["Hotel Name"][r] = response["results"][0]["name"]
hotel_df.head()

C:\Users\krist\AppData\Local\Temp\ipykernel_30840\1839956463.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"][r] = response["results"][0]["name"]


IndexError: list index out of range

In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [35]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations=locations)
heatmap_humidity.add_layer(markers)
# Display figure
heatmap_humidity

Figure(layout=FigureLayout(height='420px'))